# Andrew Kim

# ADS 509-01-SU23: Applied Text Mining

# Assignment 4.1: Text Classification Model

# 6/5/2023

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
import collections
from collections import Counter, defaultdict

# Additional packages imported
import pandas as pd
import tokenize
import re
import string
from string import punctuation

# Feel free to include your text patterns functions
# from text_functions_solutions import clean_tokenize, get_patterns
from functions.text_functions_solutions import clean_tokenize

In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.

In [3]:
convention_data = []

# fill this list up with items that are themselves lists. 
# The first element in the sublist should be the cleaned and tokenized
# text in a single string. 
# The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT * FROM conventions;
                            ''')

for row in query_results :
    # store the results in convention_data
    convention_data.append([clean_tokenize(row[5]), row[0]])
    

Let's look at some random entries and see if they look right. 

In [4]:
random.choices(convention_data,k=10)

[['he’ll listen he’ll need', 'Democratic'],
 ['i’d gone bed david received call lee’s alarm company front door pawn shop breached time didn’t wake tell probably knew would tried stop insist going slept looters ransacking shop shot killed dave cold blood livestreamed execution last moments earth david’s grandson watching video facebook real time realizing watching grandfather dying sidewalk',
  'Republican'],
 ['foreign language 001635 greetings good evening name derrick lente state representative coming homeland ancestors beautiful pueblo sandia new mexico diverse state home 23 indigenous sovereign nations rich multicultural history united love beautiful place call home believe owe next generation protect natural cultural resources inheritance end also respect tribal sovereignty new mexico proudly casts 4 votes bernie sanders 42 votes next president united states america joe biden',
  'Democratic'],
 ['okay', 'Democratic'],
 ['let’s get real there’s lot riding election', 'Democratic'],

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [5]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [6]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    ret_dict = dict()

    unique_tokens = set(text.split())

    for token in unique_tokens:
        if token in fw:
            ret_dict[token] = True
    
    return ret_dict

In [7]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [8]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [9]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [10]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [11]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Response: __Based off the classifier and from a general perspective, the list that contains the most informative features correlates to today's society and the current issues that impose potential threats or future uncertainties within everyday life. These are features that have been and continue to be evoked or mentioned between either or both parties.__

__From an analytical perspective, I found it both interesting and odd that 92% of the most informative features leans more with the Republican Party, with the Democratic Party only receiving the terms "votes" and "climate" as their key importative features. While it would make sense that the term "climate" would be emphasized more from the Democratic Party, I found it odd that the term "liberal" leans more towards the Republican Party when I initially perceived "liberal" to be more emphasized from the Democratic Party. In general, I sense a huge disparity or imbalance within each feature being more directed towards the Republican Party than the Democratic Party. Therefore, the possibility of the data being imbalanced or even biased cannot be ruled out.__

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [12]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [13]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [14]:
# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

punctuation = set(punctuation)
tw_punct = punctuation - {"#"}

tweet_data = []

for row in results :
    tweet_data.append([clean_tokenize(row[2], tw_punct), row[1]])


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [15]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [16]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet,tweet))
    # Fill in the right-hand side above with code that estimates the actual party

    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe #rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans #trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater #kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1h

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [17]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0

num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet,tweet))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [18]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3695, 'Democratic': 583}),
             'Democratic': defaultdict(int,
                         {'Republican': 4817, 'Democratic': 907})})

### Reflections

Response: __The results here appear identical from the first section that generated the most informative features, with the Republican Party being more favored than the Democratic Party. Therefore, this validates a huge presence of potential bias or imbalance within the given data that needs to be addressed.__